In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append(os.path.abspath('../src'))

In [2]:
from adversarial_validation import Validator
from data_loader import DiabetesLoader
from sklearn.pipeline import Pipeline
from preprocessing import global_preprocessor
from metric import evaluate_model

loader = DiabetesLoader('../data/raw/train.csv')
loader.load_external('../data/external/diabetes_dataset.csv')

X,y = loader.get_full()
pipeline = global_preprocessor()

X =  pipeline.fit_transform(X)

validator = Validator(X, 'is_external')
mask = validator.get_mask()

X_masked = X[mask]
y_masked = y[mask]

print(f"After filtered x: {X_masked.shape}, y: {y_masked.shape}")

Loading data from /app/data/raw/train.csv
Data Loaded. Shape: (700000, 25)
loading external data from ../data/external/diabetes_dataset.csv
Training Adversarial Validator (XGB)
After filtered x: (756886, 25), y: (756886,)


In [ ]:
from config import XGB_PARAMS
from preprocessing import get_xgb_preprocessor
from xgboost import XGBClassifier

pipe_xgb = Pipeline([
    ('preprocessor', get_xgb_preprocessor()),
    ('model', XGBClassifier(**XGB_PARAMS))
])

res_xgb = evaluate_model(pipe_xgb, X_masked, y_masked)

Starting CV on Pipeline...
Fold 1: 0.7253
Fold 2: 0.7238
Fold 3: 0.7244
Fold 4: 0.7257
Fold 5: 0.7257
Result: 0.72496 ± 0.00074 (Time: 71.9s)


: 

In [ ]:
from preprocessing import add_important_interaction

X_masked_interact = add_important_interaction(X_masked, X_masked.columns.tolist())

res_xgb_interact = evaluate_model(pipe_xgb, X_masked_interact, y_masked)

combining 15 cols


/app/src/preprocessing.py:120: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[f"{col1}_x_{col2}"] = df[col1] * df[col2]
/app/src/preprocessing.py:120: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[f"{col1}_x_{col2}"] = df[col1] * df[col2]
/app/src/preprocessing.py:120: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use 

added 105 cols as interactions
